In [4]:
! pip install -q elevenlabs dotenv

In [1]:
from elevenlabs import ElevenLabs
import os
from dotenv import load_dotenv
load_dotenv()

'sk_31b3514621978acb768d4247335d12d275b2bd644decbb3a'

In [4]:

client = ElevenLabs(
    api_key=os.getenv("ELEVENLABS_API_KEY")
,
)
client.conversational_ai.conversations.list().conversations

[ConversationSummaryResponseModel(agent_id='agent_01jzr6wjqef188s5dr4wd9hczq', agent_name='Mindfulness coach', conversation_id='conv_01jzxnh0e9eqe95kx7db0jy1tp', start_time_unix_secs=1752268571, call_duration_secs=60, message_count=0, status='failed', call_successful='unknown'),
 ConversationSummaryResponseModel(agent_id='agent_01jzr6wjqef188s5dr4wd9hczq', agent_name='Mindfulness coach', conversation_id='conv_01jzxnh0e8e5xs4dmzq099enek', start_time_unix_secs=1752268571, call_duration_secs=60, message_count=0, status='failed', call_successful='unknown'),
 ConversationSummaryResponseModel(agent_id='agent_01jzr6wjqef188s5dr4wd9hczq', agent_name='Mindfulness coach', conversation_id='conv_01jzw0swveec9v1m0dt1yzn9b4', start_time_unix_secs=1752213287, call_duration_secs=4, message_count=1, status='failed', call_successful='failure'),
 ConversationSummaryResponseModel(agent_id='agent_01jzr6wjqef188s5dr4wd9hczq', agent_name='Mindfulness coach', conversation_id='conv_01jzw0sp36fwgbvnsv8sfgk6rk',

In [ ]:
import requests

url = "https://api.elevenlabs.io/v1/convai/conversations/conv_01jzrmxncted7tg4w24b994v4z/audio"

headers = {"xi-api-key": os.getenv("ELEVENLABS_API_KEY")}

response = requests.get(url, headers=headers)

response.json()

In [18]:
client.conversational_ai.conversations.get(
    conversation_id="conv_01jzrmxncted7tg4w24b994v4z",
).transcript

[ConversationHistoryTranscriptCommonModelOutput(role='agent', message='Hi babe. What is on the heart and mind right now?', tool_calls=[], tool_results=[], feedback=None, llm_override=None, time_in_call_secs=0, conversation_turn_metrics=None, rag_retrieval_info=None, llm_usage=None, interrupted=False, original_message=None, source_medium=None),
 ConversationHistoryTranscriptCommonModelOutput(role='user', message="I'm thinking about a really difficult situation like being vulnerable and getting no response back or being told by someone I love that they just want to be platonic friends. And can you help me simulate this scenario in a way that's very emotionally present?", tool_calls=[], tool_results=[], feedback=None, llm_override=None, time_in_call_secs=3, conversation_turn_metrics=None, rag_retrieval_info=None, llm_usage=None, interrupted=False, original_message=None, source_medium='audio'),
 ConversationHistoryTranscriptCommonModelOutput(role='agent', message="Oh, sweetie, that's a tou

In [20]:
# Test the journal backend integration
import requests
import json
from datetime import datetime

# Test the journal API endpoints
backend_url = "http://localhost:8000"
conversation_id = f"test_conv_{int(datetime.now().timestamp())}"

# Test saving journal content
journal_data = {
    "conversation_id": conversation_id,
    "content": "This is a test journal entry with some <strong>bold</strong> text.",
    "timestamp": datetime.now().isoformat()
}

# Test transcript entry
transcript_data = {
    "conversation_id": conversation_id,
    "text": "Hello, this is a test transcript",
    "speaker": "user",
    "timestamp": datetime.now().isoformat()
}

print(f"Testing with conversation ID: {conversation_id}")

try:
    # Test journal save
    response = requests.post(f"{backend_url}/api/journal/{conversation_id}", json=journal_data)
    print(f"Journal save response: {response.status_code} - {response.json()}")
    
    # Test transcript save
    response = requests.post(f"{backend_url}/api/transcript/{conversation_id}", json=transcript_data)
    print(f"Transcript save response: {response.status_code} - {response.json()}")
    
    # Test journal retrieval
    response = requests.get(f"{backend_url}/api/journal/{conversation_id}")
    print(f"Journal retrieve response: {response.status_code}")
    print(f"Journal data: {json.dumps(response.json(), indent=2)}")
    
except Exception as e:
    print(f"Error testing backend: {e}")
    print("Make sure the backend server is running on http://localhost:8000")

Testing with conversation ID: test_conv_1752273161
Journal save response: 200 - {'success': True}
Transcript save response: 200 - {'success': True}
Journal retrieve response: 200
Journal data: {
  "content": "This is a test journal entry with some <strong>bold</strong> text.",
  "transcript": [
    {
      "text": "Hello, this is a test transcript",
      "speaker": "user",
      "timestamp": "2025-07-11T15:32:41.231772"
    }
  ],
  "created_at": "2025-07-11T15:32:41.250484",
  "updated_at": "2025-07-11T15:32:41.231772"
}


In [4]:
# Debug the actual transcript structure from ElevenLabs
import requests
import os
import json
from dotenv import load_dotenv
load_dotenv()

# Get a recent conversation to see the exact structure
conversation_id = "conv_01k05j10jwfv49a9hf0pyxbvvv"  # Use the one from the logs
url = f"https://api.elevenlabs.io/v1/convai/conversations/{conversation_id}"
headers = {"xi-api-key": os.getenv("ELEVENLABS_API_KEY")}

print(f"Debugging conversation: {conversation_id}")
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    print(f"✅ Retrieved conversation data ({len(str(data))} characters)")
    print(f"📋 Top-level keys: {list(data.keys())}")
    
    # Focus on transcript structure
    if 'transcript' in data:
        transcript = data['transcript']
        print(f"\n🎤 Transcript type: {type(transcript)}")
        print(f"🎤 Transcript length: {len(transcript) if isinstance(transcript, list) else 'Not a list'}")
        
        if isinstance(transcript, list) and len(transcript) > 0:
            print("\n📝 First few transcript entries:")
            for i, entry in enumerate(transcript[:3]):
                print(f"\nEntry {i+1}:")
                print(f"  Type: {type(entry)}")
                if isinstance(entry, dict):
                    print(f"  Keys: {list(entry.keys())}")
                    # Show the actual structure
                    print(f"  Full entry: {json.dumps(entry, indent=4)}")
                else:
                    print(f"  Value: {entry}")
    
    # Also check if there are other conversation-related fields
    print(f"\n🔍 Other potential conversation fields:")
    for key in data.keys():
        if 'conv' in key.lower() or 'message' in key.lower() or 'turn' in key.lower():
            print(f"  {key}: {type(data[key])}")
            
else:
    print(f"❌ Error: {response.status_code} - {response.text}")

Debugging conversation: conv_01k05j10jwfv49a9hf0pyxbvvv
✅ Retrieved conversation data (4455 characters)
📋 Top-level keys: ['agent_id', 'conversation_id', 'status', 'user_id', 'transcript', 'metadata', 'analysis', 'conversation_initiation_client_data', 'has_audio', 'has_user_audio', 'has_response_audio']

🎤 Transcript type: <class 'list'>
🎤 Transcript length: 3

📝 First few transcript entries:

Entry 1:
  Type: <class 'dict'>
  Keys: ['role', 'message', 'tool_calls', 'tool_results', 'feedback', 'llm_override', 'time_in_call_secs', 'conversation_turn_metrics', 'rag_retrieval_info', 'llm_usage', 'interrupted', 'original_message', 'source_medium']
  Full entry: {
    "role": "agent",
    "message": "What is on the heart and mind right now?",
    "tool_calls": [],
    "tool_results": [],
    "feedback": null,
    "llm_override": null,
    "time_in_call_secs": 0,
    "conversation_turn_metrics": null,
    "rag_retrieval_info": null,
    "llm_usage": null,
    "interrupted": false,
    "origi

In [5]:
# Examine transcript structure for frontend parsing
print("=== TRANSCRIPT STRUCTURE ANALYSIS ===")
print(f"Transcript type: {type(transcript)}")
print(f"Transcript length: {len(transcript)}")
print()

# Look at first few entries
for i, entry in enumerate(transcript[:3]):
    print(f"Entry {i}:")
    print(f"  Type: {type(entry)}")
    if isinstance(entry, dict):
        print(f"  Keys: {list(entry.keys())}")
        for key, value in entry.items():
            print(f"    {key}: {repr(value)[:100]}...")
    print()

# Get all unique keys
all_keys = set()
for entry in transcript:
    if isinstance(entry, dict):
        all_keys.update(entry.keys())

print(f"All unique keys: {sorted(all_keys)}")

# Show a typical entry structure
if transcript:
    print(f"\nSample entry structure:")
    sample = transcript[0]
    print(f"Keys: {list(sample.keys())}")
    for key in ['speaker', 'role', 'message', 'text', 'content', 'user_id', 'agent_id']:
        if key in sample:
            print(f"  {key}: {repr(sample[key])}")

=== TRANSCRIPT STRUCTURE ANALYSIS ===
Transcript type: <class 'list'>
Transcript length: 3

Entry 0:
  Type: <class 'dict'>
  Keys: ['role', 'message', 'tool_calls', 'tool_results', 'feedback', 'llm_override', 'time_in_call_secs', 'conversation_turn_metrics', 'rag_retrieval_info', 'llm_usage', 'interrupted', 'original_message', 'source_medium']
    role: 'agent'...
    message: 'What is on the heart and mind right now?'...
    tool_calls: []...
    tool_results: []...
    feedback: None...
    llm_override: None...
    time_in_call_secs: 0...
    conversation_turn_metrics: None...
    rag_retrieval_info: None...
    llm_usage: None...
    interrupted: False...
    original_message: None...
    source_medium: None...

Entry 1:
  Type: <class 'dict'>
  Keys: ['role', 'message', 'tool_calls', 'tool_results', 'feedback', 'llm_override', 'time_in_call_secs', 'conversation_turn_metrics', 'rag_retrieval_info', 'llm_usage', 'interrupted', 'original_message', 'source_medium']
    role: 'user'..